In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### 1. Import Libraries

In [0]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import os
from numpy.random import choice as npc
import numpy as np
import time
import random
import torchvision.datasets as dset
from PIL import Image
import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as dset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import sys
from collections import deque

### 2. Build Network

In [3]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        self.buildNetwork()


    def buildNetwork(self):
        """
            Declare layes of the networks
        """
        self.convLayers = nn.Sequential(
          nn.Conv2d(in_channels= 1, out_channels= 64, kernel_size=10),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(2), 
          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7), 
          nn.ReLU(),              
          nn.MaxPool2d(2),
          nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),
          nn.ReLU(), 
          nn.MaxPool2d(2),
          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),
          nn.ReLU() 
        )
        self.linearLayers = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.Sigmoid())
        self.outLayer = nn.Linear(4096, 1)



    def forward(self,x1,x2):
        """
            x1, x2 : Two input images 
        """

        # Perform two forward pass from convolution layer for each input images
        x1_out = self.convLayers(x1)
        x2_out = self.convLayers(x2)
        
        # Resize both outputs for linear layers
        x1_out = x1_out.view(x1_out.size()[0], -1)
        x2_out = x2_out.view(x2_out.size()[0], -1)
        
        x1_out = self.linearLayers(x1_out)
        x2_out = self.linearLayers(x2_out)

        # Take absolute distance for both outputs
        distance = torch.abs(x1_out - x2_out)
        output = self.outLayer(distance)
        
        return output

test = Network()
print(test)
  

Network(
  (convLayers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(10, 10), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1))
    (10): ReLU()
  )
  (linearLayers): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): Sigmoid()
  )
  (outLayer): Linear(in_features=4096, out_features=1, bias=True)
)


### 3. Import Dataset

In [4]:
%%shell
git clone https://github.com/brendenlake/omniglot.git
cd omniglot/python
unzip -q images_evaluation.zip
unzip -q images_background.zip
cd ../..
# setup directory for saving models
mkdir models

Cloning into 'omniglot'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (81/81), done.


### 4. Dataset Class

In [0]:
class DataSet(Dataset):

    def __init__(self, train=True ,times=200 ,batch_size=20, transform=None):
        super().__init__()
        self.train= train
        self.transform = transform
        self.times = times
        self.batch_size = batch_size
        self.img1 = None
        self.c1 = None
        self.datas = {}
        self.num_classes = 0
        self.train_dataPath = "./omniglot/python/images_background"
        self.test_dataPath = "./omniglot/python/images_evaluation"
        if self.train:
          self.loadTrainData()
        else:
          self.loadTestData()




    def __len__(self):
        """
          Returns count of total instances
        """
        if self.train:
            return 21000000
        else:
            return self.times * self.batch_size



    def loadTrainData(self):
      """
          Loads training dataset from self.train_dataPath 
          Counts the number of classes
          Maintains images wrt classes in dictonary
      """

      classesConut = 0
      annotationAngels  = [0, 90, 180, 270]
      for i in annotationAngels:
        for j in os.listdir(self.train_dataPath):
            for k in os.listdir(os.path.join(self.train_dataPath, j)):
                self.datas[classesConut] = []
                for m in os.listdir(os.path.join(self.train_dataPath, j, k)):
                    filePath = os.path.join(self.train_dataPath, j, k, m)
                    # Rotate grayscale images along angles and store in dictionary
                    self.datas[classesConut].append(Image.open(filePath).rotate(i).convert('L'))
                classesConut += 1
      print("Train Data Loaded")
      self.num_classes =  classesConut




    def getTrainImages(self,index):
        """
          Gets training images at 'index'
          Return : two images and label
        """
        label = None
        if index % 2 == 1:      # Same images 
            label = 1.0
            randomIndexA = random.randint(0, self.num_classes - 1)
            image1 = random.choice(self.datas[randomIndexA])
            image2 = random.choice(self.datas[randomIndexA])
        else:
            label = 0.0         # Different images
            randomIndexA = random.randint(0, self.num_classes - 1)
            randomIndexB = random.randint(0, self.num_classes - 1)
            # While both random int are equal repeat
            while randomIndexA == randomIndexB:
                randomIndexB = random.randint(0, self.num_classes - 1)
            image1 = random.choice(self.datas[randomIndexA])
            image2 = random.choice(self.datas[randomIndexB])

        # Apply transfrom on images if any
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
        return image1,image2,label



    def __getitem__(self,index):
        """
          Returns items at index
        """
        if self.train:
            image1,image2,label = self.getTrainImages(index)
            return image1, image2, torch.from_numpy(np.array([label], dtype=np.float32))
        else:

            img1,img2 = self.getTestImages(index)
            return img1, img2



    def getTestImages(self,index):
      """
          Gets testing images at 'index'
          Return : two images
      """
      randomIndex = index % self.batch_size
      label = None
      if randomIndex == 0:
          self.c1 = random.randint(0, self.num_classes - 1)
          self.img1 = random.choice(self.datas[self.c1])
          img2 = random.choice(self.datas[self.c1])
      else:
          randomClass = random.randint(0, self.num_classes - 1)
          while self.c1 == randomClass:
              randomClass = random.randint(0, self.num_classes - 1)
          img2 = random.choice(self.datas[randomClass])
      if self.transform:
          img1 = self.transform(self.img1)
          img2 = self.transform(img2)
      return img1,img2



    def loadTestData(self):
      """
          Loads testing dataset from self.test_dataPath 
          Counts the number of classes
          Maintains images wrt classes in dictonary
      """
      
      classCount = 0
      for i in os.listdir(self.test_dataPath):
        for j in os.listdir(os.path.join(self.test_dataPath, i)):
            self.datas[classCount] = []
            for k in os.listdir(os.path.join(self.test_dataPath, i, j)):
                filePath = os.path.join(self.test_dataPath, i, j, k)
                self.datas[classCount].append(Image.open(filePath).convert('L'))
            classCount += 1
      print("Test Data Loaded")
      self.num_classes =  classCount



### Execution

In [10]:
class OneShotLearner():
  def __init__(self):
    self.transformations = transforms.Compose([
        transforms.RandomAffine(15),
        transforms.ToTensor()
    ])
    self.workers = 4                  # Number of dataloader workers
    self.batch_size = 128             # Training Batch size
    self.test_batch_size = 20         # Testing Batch size
    self.times = 400
    self.CUDA = True
    self.learning_rate = 0.1
    self.max_iterations = 35000
    self.model_path = 'models'        # Path to save weights
    self.train_dataset = DataSet(train=True,transform=self.transformations)
    self.train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.workers)
    self.test_dataset = DataSet( train=False, transform=transforms.ToTensor(), times=self.times, batch_size=self.test_batch_size)
    self.test_loader = DataLoader(self.test_dataset, batch_size=self.test_batch_size, shuffle=False, num_workers=self.workers)
    self.model = Network()
    self.cudaTrainSetting()
    self.loss_fn = torch.nn.BCEWithLogitsLoss(size_average=True)

    
  def cudaTrainSetting(self):
    if self.CUDA:
      self.model.cuda()
    self.model.train()

  def load_weights(self, PATH):
    self.model.load_state_dict(torch.load(PATH))
    self.model.eval()

  def train(self):
    optimizer = torch.optim.SGD(self.model.parameters(),lr = self.learning_rate )
    optimizer.zero_grad()
    train_loss = []
    loss_val = 0
    time_start = time.time()

    for batch_id, (img1, img2, label) in enumerate(self.train_loader, 1):
        if batch_id > self.max_iterations:
            break
        if self.CUDA:
            img1, img2, label = Variable(img1.cuda()), Variable(img2.cuda()), Variable(label.cuda())
        else:
            img1, img2, label = Variable(img1), Variable(img2), Variable(label)
        optimizer.zero_grad()
        output = self.model.forward(img1, img2)     # Forward Pass
        loss = self.loss_fn(output, label)          # Calculate loss 
        loss_val += loss.item()
        loss.backward()                             # BackPropogation
        optimizer.step()                            # Update weights

        # Display loss
        if batch_id % 10 == 0:
            print('Batch Id=> %d\tloss=>\t%.5f\ttime lapsed=>\t%.2f s'%(batch_id, loss_val/10, time.time() - time_start))
            loss_val = 0
            time_start = time.time()

        # Save weights
        if batch_id % 200 == 0:
            self.predict(batch_id)
            torch.save(self.model.state_dict(), self.model_path + '/model-inter-' + str(batch_id+1) + ".pt")
        train_loss.append(loss_val)
  
  
  def predict(self,batch_id):        
    right, error = 0, 0
    for _, (test1, test2) in enumerate(self.test_loader, 1):
        if self.CUDA:
            test1, test2 = test1.cuda(), test2.cuda()
        test1, test2 = Variable(test1), Variable(test2)
        output = self.model.forward(test1, test2).data.cpu().numpy()
        pred = np.argmax(output)
        if pred == 0:
            right += 1
        else: 
            error += 1
    print('-'*100)
    print('Batch Id=> %d\tcorrect=>\t%d\terror=>\t%d\tprecision=>\t%f'%(batch_id, right, error, right*1.0/(right+error)))
    print('-'*100)

"""
  Execute training
"""

test = OneShotLearner()
test.train()

#PATH = 'drive/My Drive/Deep_Project/model-inter-35001.pt'
#test.load_weights(PATH)
test.predict(1)


Train Data Loaded
Test Data Loaded


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


----------------------------------------------------------------------------------------------------
Batch Id=> 1	correct=>	382	error=>	18	precision=>	0.955000
----------------------------------------------------------------------------------------------------


In [0]:
test.times = 9000
test.test_dataset = DataSet( train=False, transform=transforms.ToTensor(), times=test.times, batch_size=test.test_batch_size)
test.test_loader = DataLoader(test.test_dataset, batch_size=test.test_batch_size, shuffle=False, num_workers=test.workers)      
test.predict(100)

In [0]:
print("final accuracy: ", acc)

### Team
1. Muhammad Nuaraiz Mushtaq  - 160106
2. Govinda Kumar             - 160249
3. Abid Waqar                - 160229
4. Ahsan Abbas               - 160101